In [ ]:
import numpy as np
import os
import tensorflow as tf
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
import glob
import random
import time
from tqdm import tqdm_notebook
from lxml import etree
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle

%matplotlib inline

In [ ]:
def load_model(file_path):
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(file_path, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
    return detection_graph
            
def load_image(image_path):
    image = Image.open(image_path)
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

def run_inference(sess, ops, image_tensor, image):
    output_dict = {}
    
    time_s = time.time()
    num_detections, boxes, scores, classes = sess.run(ops, feed_dict={image_tensor: image})
    time_t = time.time() - time_s
    
    output_dict['num_detections'] = int(num_detections[0])
    output_dict['detection_classes'] = classes[0].astype(np.uint8)
    output_dict['detection_boxes'] = boxes[0]
    output_dict['detection_scores'] = scores[0]
    output_dict['detection_time'] = time_t
    
    return output_dict

In [ ]:
def extract_class(detection_dict, threshold = 0.5):
    detection_classes = detection_dict['detection_classes']
    detection_scores = detection_dict['detection_scores']
    if detection_scores[0] >= threshold:
        return detection_classes[0]
    return 0

def load_labels(image_paths, labels_dir, label_map):
    labels = []
    for image_path in image_paths:
        file_name = os.path.splitext(os.path.basename(image_path))[0]
        image_label = os.path.join(labels_dir, file_name + '.xml')
        image_class = 0 # Fake class for "background" images without traffic lights

        if os.path.isfile(image_label):
            with tf.gfile.GFile(image_label, 'r') as f:
                xml = f.read()
            xml = etree.fromstring(xml)
            object_el = next(xml.iter('object'), None)
            if object_el is not None:
                class_name = next(object_el.iter('name')).text
                image_class = label_map[class_name]

        labels.append(image_class)
    return labels

def evaluate(name, graph, image_paths, labels, cpu = False):
    
    predictions = []
    
    with graph.as_default():
    
        image_tensor = graph.get_tensor_by_name('image_tensor:0')
        boxes_tensor = graph.get_tensor_by_name('detection_boxes:0')
        scores_tensor = graph.get_tensor_by_name('detection_scores:0')
        classes_tensor = graph.get_tensor_by_name('detection_classes:0')
        detections_tensor = graph.get_tensor_by_name('num_detections:0')

        ops = [detections_tensor, boxes_tensor, scores_tensor, classes_tensor]
        
        config = None
        
        if cpu:
            config = tf.ConfigProto(device_count = {'GPU': 0})

        with tf.Session(config=config) as sess:
            
            if len(image_paths) > 0:
                # Preload
                image = load_image(image_paths[0])
                run_inference(sess, ops, image_tensor, np.expand_dims(image, axis=0))
            
            time_t = 0
            for image_path in tqdm_notebook(image_paths, desc='{}'.format(name), unit=' images'):
                image = load_image(image_path)
                # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                image = np.expand_dims(image, axis=0)
                # Actual detection.
                output_dict = run_inference(sess, ops, image_tensor, image)
                time_t += output_dict['detection_time']
                prediction = extract_class(output_dict)
                predictions.append(prediction)
                
    for i, (label, prediction) in enumerate(zip(labels, predictions)):
        if label != prediction:
            print('Image misclassified: {} (Expected: {}, Predicted: {})'.format(image_paths[i], label, prediction))
                
    accuracy = accuracy_score(labels, predictions)
    
    return accuracy, time_t * 1000


In [ ]:
LABELS_MAP_PATH = os.path.join('..', 'config', 'labels_map.pbtxt')
LABELS_MAP = {'Green': 1, 'Yellow': 2, 'Red': 3}

def evaluate_models(models_dir, data_dir, models, model_file = 'frozen_inference_graph.pb', n_samples = None, cpu = False):
    image_paths = glob.glob(os.path.join(data_dir, '*.jpg'))
    labels = load_labels(image_paths, os.path.join(data_dir, 'labels'), LABELS_MAP)
    image_paths, labels = shuffle(image_paths, labels, n_samples = n_samples)
    
    for model_name in models:
        model_path = os.path.join(models_dir, model_name, model_file)
        graph = load_model(model_path)
        accuracy, total_time = evaluate(model_name, graph, image_paths, labels, cpu = cpu)
        avg_time = total_time / len(image_paths)
        fps = 1000 / avg_time
        print('Model {} Accuracy: {:.3f} (Avg Time: {:.3f} ms, FPS: {:.3f})'.format(model_name, accuracy, avg_time, fps))

In [ ]:
MODELS_DIR = os.path.join('..', 'models', 'exported')
DATA_DIR = os.path.join('..', 'data', 'simulator')

n_samples = 500

models = ['ssd_mobilenet_v1', 'ssd_mobilenet_v2', 'ssdlite_mobilenet_v2', 'ssd_inception_v2']

evaluate_models(MODELS_DIR, DATA_DIR, models, n_samples = n_samples, cpu = False)

# Performance eval

n_samples = 50

evaluate_models(MODELS_DIR, DATA_DIR, models, n_samples = n_samples, cpu = False)
evaluate_models(MODELS_DIR, DATA_DIR, models, n_samples = n_samples, cpu = True)

# Performance eval on optimized models

MODELS_DIR = os.path.join('..', 'models', 'exported_optimized')

evaluate_models(MODELS_DIR, DATA_DIR, models, model_file = 'graph_optimized.pb', n_samples = n_samples, cpu = False)
evaluate_models(MODELS_DIR, DATA_DIR, models, model_file = 'graph_optimized.pb', n_samples = n_samples, cpu = True)